# Assignment 5
## Due May 23 at 12:00

Please note: 

- Read the instructions in the exercise PDF and in this notebook carefully.
- Add your solutions *only* at `YOUR CODE HERE`/`YOUR ANSWER HERE` and remove the corresponding `raise NotImplementedError()`.
- Do not chance the provided code and text, if not stated.
- Do not *add* or *delete* cells.
- Do not `import` additional functionality. 
- Before submitting: Please make sure, that your notebook can be executed from top to bottom `Menu -> Kernel -> Restart & Run all`. 

## Exercise 1
(a) First, the optimization problem has to be reformulated in standard form:

minimize $f(x,y) = -x-y$

s.t. $g(x,y) = x^2 + 2 y^2 -5 = 0$

The Lagrangian is: $L(x,y,\nu) = f(x,y) + \nu g(x,y)$

Next, set the partial derivatives of the Lagrangian to 0:

$\frac{\partial L}{\partial x} = 2\nu x - 1 \overset{!}{=} 0$

$\frac{\partial L}{\partial y} = 4\nu y - 1 \overset{!}{=} 0$

$\frac{\partial L}{\partial \nu} = x^2 + 2 y^2 -5 \overset{!}{=} 0$

$\implies x = 1.83; y = 0.91, \nu = 0.27$




## Exercise 4

In [1]:
import time 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from numpy.testing import assert_equal, assert_almost_equal

from sklearn import preprocessing
from sklearn.datasets import load_breast_cancer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

#  Hide warnings of LinearSVC, LogisticRegression

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=ConvergenceWarning)

np.random.seed(42)

## a) Know the Dataset


- **Features:** 
    YOUR ANSWER HERE
- **Labels:** 
    YOUR ANSWER HERE

In [2]:
dataset = load_breast_cancer()

xs = dataset.data
ys = dataset.target

Split `xs, ys` to the training set `xs_train, ys_train` (size $70\%$) and the test set `xs_test, ys_test` (size $30\%$) 

## b)/c) Optimize the Hyperparameters: Linear SVM vs Logistic Regression


In [3]:
# center and normalize

scaler = preprocessing.StandardScaler().fit(xs)
xs = scaler.transform(xs)

# split into training and test test
n_train = int(len(xs) * .7)

xs_train, xs_test = xs[:n_train], xs[n_train:]
ys_train, ys_test = ys[:n_train], ys[n_train:]

# evaluate standard SVM
svm_estimator = LinearSVC().fit(xs_train,ys_train)
print('We can do better than ', np.mean(svm_estimator.predict(xs_test) != ys_test))

We can do better than  0.03508771929824561


Find good hyperparameters *without* the test set.

In [4]:
# DO NOT USE xs_test, ys_test here!
# Please make your optimization reproducable (e.g. set random_state, seed, …)
# LinearSVC and LogisticRegression

best_params_svm, best_params_lr = {}, {}

### YOUR CODE HERE
raise NotImplementedError()

print(f"Cross validation errors of best model: Linear SVM {cv_errors_svm.min():.4f}, Logistic Regression {cv_errors_lr.min():.4f}")

Cross validation errors of best model: Linear SVM 0.025316, Logistic Regression 0.027848


In [5]:
# if you did not solve the cross validation, use this:
# svm_estimator = LinearSVC(C=0.0035, random_state=42).fit(xs_train, ys_train)
# lr_estimator = LogisticRegression(C=0.4037,random_state=42).fit(xs_train, ys_train)
# else, use this:
svm_estimator = best_estimator_svm
lr_estimator = best_estimator_lr

### YOUR CODE HERE
raise NotImplementedError()

Test Errors: Linear SVM 0.012, Logistic Regression 0.029


What do you observe and why?

Answer: YOUR ANSWER HERE

## d) State concerns

1. **Ethical:** 
    YOUR ANSWER HERE

2. **Technical/Statistical:**
    YOUR ANSWER HERE

3. **Any:**
    YOUR ANSWER HERE